In [1]:
import maze_library
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


2025-03-23 21:02:56.507829: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-23 21:02:56.518890: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-23 21:02:56.546289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742763776.582736  614086 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742763776.593223  614086 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742763776.626249  614086 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
def map_int_to_direction(input):
    if input == 0:
        return maze_library.Direction.North
    if input == 1:
        return maze_library.Direction.East
    if input == 2:
        return maze_library.Direction.South
    else:
        return maze_library.Direction.West

In [3]:
env = maze_library.init_environment_python(10,10)
maze_library.create_kruzkals_maze(env)
maze_library.make_maze_imperfect(env)

In [4]:
def action_result_to_tuple(action_result):
    """
    Converts an ActionResult object into a tuple.

    Args:
        action_result (ActionResult): The result of an action in the environment.

    Returns:
        tuple: (observation, reward, done, truncated, info)
    """
    observation = action_result.observation  # The new observation after taking action
    reward = action_result.reward  # Reward received
    done = action_result.is_done  # Whether the episode is finished
    truncated = action_result.is_truncated  # Whether the episode was truncated
    info = action_result.info # Additional info

    return observation, reward, done, truncated, info


In [5]:
def action_result_to_input_vector(state, info):
    available_paths = state.available_paths
    return np.array([
        available_paths.get(maze_library.Direction.North, 0),
        available_paths.get(maze_library.Direction.East, 0),
        available_paths.get(maze_library.Direction.South, 0),
        available_paths.get(maze_library.Direction.West, 0),
        state.current_location[0] / 10,
        state.current_location[1] / 10,
        state.end_node[0] / 10,
        state.end_node[1] / 10,
        info.manhattan_distance / 10,
        info.goal_dx/5,
        info.goal_dy/5,
        info.visited_node
    ])

In [6]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [12]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="elu", input_shape=input_shape),
    tf.keras.layers.Dense(32, activation="elu"),
    tf.keras.layers.Dense(n_outputs)
])

/Development/FYP/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-23 21:03:00.840951: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
def epsilon_greedy_policy(flat_state, epsilon):
    valid_actions = [i for i, steps in enumerate(flat_state[:4]) if steps > 0]
    if np.random.rand() < epsilon:
        # Choose only from valid actions
        return np.random.choice(valid_actions)
    else:
        # Model-based action selection (only among valid actions)
        q_values = model.predict(flat_state.reshape(1, -1),  verbose=0)[0]
        return max(valid_actions, key=lambda a: q_values[a])  # Highest Q-value among valid actions


In [8]:
from collections import deque

replay_buffer = deque(maxlen=2000)

In [9]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    return [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(6)
    ]  # [states, actions, rewards, next_states, dones, truncateds]

In [10]:
def play_one_step(env, state, info, epsilon):
    flat_state = action_result_to_input_vector(state, info)
    action = epsilon_greedy_policy(flat_state, epsilon)
    action_obj = maze_library.create_action(map_int_to_direction(action))
    next_state, reward, done, truncated, new_info = action_result_to_tuple(env.take_action(action_obj))
    flat_next_state = action_result_to_input_vector(next_state, new_info)
    replay_buffer.append((flat_state, action, reward, flat_next_state, done, truncated))
    reward = reward 
    return next_state, reward, done, truncated, info

In [11]:
np.random.seed(42)
tf.random.set_seed(42)
rewards = [] 
highest_reward = -1000000

In [12]:
batch_size = 32
discount_factor = 0.95
optimizer = tf.keras.optimizers.Nadam(learning_rate=1e-2)
loss_fn = tf.keras.losses.MeanSquaredError()  # Use the class
def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones, truncateds = experiences
    next_Q_values = model.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + runs * discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [13]:
maze_library.print_weighted_graph(env)

{(5, 5): {South: 1, West: 1, East: 1}, (2, 0): {East: 1, South: 1, West: 1}, (4, 3): {North: 1}, (8, 3): {West: 1, South: 1}, (2, 8): {North: 1, South: 1, East: 1}, (8, 5): {North: 1, South: 1, East: 1}, (1, 4): {North: 1, East: 1}, (0, 5): {East: 1, North: 2, South: 2}, (0, 3): {South: 2, North: 2, East: 1}, (0, 9): {North: 1}, (1, 0): {East: 1}, (1, 5): {South: 2, West: 1}, (2, 7): {West: 1, South: 1, North: 1}, (7, 3): {East: 1, North: 2, West: 2}, (4, 1): {North: 1, East: 1, West: 1}, (8, 8): {West: 1, South: 1}, (6, 9): {East: 1, West: 1, North: 1}, (7, 6): {North: 1, South: 1, East: 1}, (6, 5): {South: 1, West: 1}, (6, 1): {South: 1, North: 1, East: 1}, (9, 2): {South: 2, West: 1, North: 2}, (6, 0): {South: 1, West: 1}, (1, 8): {North: 1, West: 1, South: 1}, (2, 9): {North: 1, West: 1}, (7, 0): {South: 1}, (8, 2): {East: 1}, (2, 4): {East: 1, West: 1, North: 2}, (5, 3): {East: 2, North: 1}, (2, 6): {South: 1, North: 1, East: 1}, (2, 5): {East: 1, South: 1}, (5, 0): {South: 1, Eas

In [ ]:
steps = 500

for episode in range(100):
    result = env.reset()   
    obs, reward, done, truncated, info = action_result_to_tuple(result)
    cumilative_reward = 0
    for step in range(steps):
        epsilon = max(1 - episode / 75, 0.01)
        obs, reward, done, truncated, info = play_one_step(env, obs, info, epsilon)
        cumilative_reward += reward
        if done or truncated:
            break

    # Extra debug information
    print(f"\rEpisode: {episode}, Steps: {step}, eps: {epsilon:.3f}, reward = {cumilative_reward}")
    rewards.append(cumilative_reward)
    
    if cumilative_reward >= highest_reward:
        best_weights = model.get_weights()
        highest_reward = cumilative_reward

    if episode > 30:
        training_step(batch_size)

    with open(f'../mazeLogs/solve{episode}.json', 'w') as file:
        file.write(env.to_json_python())

    
        
model.set_weights(best_weights)  # extra code – restores the best model weights

Episode: 1, Steps: 270, eps: 1.000, reward = -208
Episode: 2, Steps: 244, eps: 0.987, reward = -194
Episode: 3, Steps: 279, eps: 0.973, reward = -9
Episode: 4, Steps: 268, eps: 0.960, reward = -251
Episode: 5, Steps: 231, eps: 0.947, reward = -82
Episode: 6, Steps: 276, eps: 0.933, reward = -235
Episode: 7, Steps: 243, eps: 0.920, reward = -106
Episode: 8, Steps: 69, eps: 0.907, reward = 1228
Episode: 9, Steps: 163, eps: 0.893, reward = 1016
Episode: 10, Steps: 271, eps: 0.880, reward = -157
Episode: 11, Steps: 265, eps: 0.867, reward = -332
Episode: 12, Steps: 87, eps: 0.853, reward = 1192
Episode: 13, Steps: 169, eps: 0.840, reward = 969
Episode: 14, Steps: 255, eps: 0.827, reward = -212
Episode: 15, Steps: 274, eps: 0.813, reward = -370
Episode: 16, Steps: 15, eps: 0.800, reward = 1110
Episode: 17, Steps: 127, eps: 0.787, reward = 1086
Episode: 18, Steps: 199, eps: 0.773, reward = 872
Episode: 19, Steps: 17, eps: 0.760, reward = 1114
Episode: 20, Steps: 282, eps: 0.747, reward = -38

In [ ]:
# extra code – this cell generates and saves Figure 18–10
plt.figure(figsize=(8, 4))
plt.plot(rewards)
plt.xlabel("Episode", fontsize=14)
plt.ylabel("Sum of rewards", fontsize=14)
plt.grid(True)
plt.show()